In [ ]:
### Configuration

MODEL_PATH = "nottino.pth"

In [ ]:
### Setup

from blinkstick import blinkstick
from jetcam.csi_camera import CSICamera
from time import sleep, time
import PIL.Image

import torch
import torchvision
import torch.nn.functional as F
import torchvision.transforms as transforms

camera = CSICamera(width=224, height=224, capture_device=0)
bs = blinkstick.find_first()

device = torch.device('cuda')

In [ ]:
### Prepare model

model = torchvision.models.resnet18(pretrained=True)
model.fc = torch.nn.Linear(512, 2)
model = model.to(device)

model.load_state_dict(torch.load(MODEL_PATH))
model.eval()

ASLEEP = 0
AWAKE = 1

TURN_OFF = 0
TURN_ON = 1

ON_WAIT = 2
OFF_WAIT = 10
MORPH_TIME = 2000

COLOR = "orange"

def current_situation():
    mean = torch.Tensor([0.485, 0.456, 0.406]).cuda()
    std = torch.Tensor([0.229, 0.224, 0.225]).cuda()

    device = torch.device('cuda')
    r = PIL.Image.fromarray(image)
    r = transforms.functional.to_tensor(r).to(device)
    r.sub_(mean[:, None, None]).div_(std[:, None, None])
    r = r[None, ...]

    output = F.softmax(model(r), dim=1).detach().cpu().numpy().flatten().argmax()
    return output

In [ ]:
### Instrumentation

from ipywidgets import Image
from IPython.display import display
from jetcam.utils import bgr8_to_jpeg
from imutils import rotate

widget = Image(format='jpeg')
image = camera.read()

def update(change):
    global image
    image = change['new']
    widget.value = bgr8_to_jpeg(rotate(image, 180))

camera.observe(update, names='value')
camera.running = True

display(widget)

In [ ]:
### Store edge timestamps to avoid acting too quickly

last_change = None
last_action = None

def tick(w):
    global last_change, last_action
    
    if w == ASLEEP:
        print(f"do asleep, last={last_action}, change={last_change}")
        if (last_action != TURN_OFF) and (last_change == None or (time() - last_change) > OFF_WAIT):
            bs.morph(name="black", duration=MORPH_TIME, steps=128)
            last_change = time()
            last_action = TURN_OFF
            
    if w == AWAKE:
        print(f"do awake, last={last_action}, change={last_change}")
        if (last_action != TURN_ON) and (last_change == None or (time() - last_change) > ON_WAIT):
            bs.morph(name=COLOR, duration=MORPH_TIME, steps=128)
            last_change = time()
            last_action = TURN_ON

In [ ]:
### Work

while True:
    try:
        tick(current_situation())
        sleep(1)
        
    except KeyboardInterrupt:
        break

In [ ]:
### Teardown

bs.turn_off()

camera.unobserve(update, names='value')
camera.running = False
camera.cap.release()